# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Before get the data, let's download all the dependencies we need

In [3]:
# Copy from Lab Segmenting and Clustering Neighborhoods in New York City
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1. Download the data

In [4]:
#downloard Cana Postal Codes from Wikipedia
Wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #use link from Task
source = requests.get(Wikiurl).text #get link
CAN_data = BeautifulSoup(source, 'lxml')

# 2. Create Dateframe

In [6]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood'] #for part1 we use only 3 columns
TorontoCity = pd.DataFrame(columns = column_names)

# Then let's loop through the data and fill the dataframe one row at a time
content = CAN_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    TorontoCity = TorontoCity.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
TorontoCity = TorontoCity[TorontoCity.Borough!='Not assigned']
TorontoCity = TorontoCity[TorontoCity.Borough!= 0]
TorontoCity.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,TorontoCity.shape[0]):
    if TorontoCity.iloc[i][2] == 'Not assigned':
        TorontoCity.iloc[i][2] = TorontoCity.iloc[i][1]
        i = i+1
                                 
df = TorontoCity.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Quickly examine the resulting dataframe.
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 3. Clean Dateframe
(If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park)

In [12]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')


In [13]:
print(df.shape)
df.head(15)

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Thank you for grading,

See you oin Part2